<a href="https://colab.research.google.com/github/JSJeong-me/Local-LLM/blob/main/05-gemma_7b-it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install hf_transfer

https://huggingface.co/google/gemma-7b-it

In [ ]:
!nvidia-smi

In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os


#os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [5]:
torch.set_default_device('cuda')

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", # 'google/gemma-2b-it
                                             quantization_config=quantization_config,
                                             low_cpu_mem_usage=True,
                                             torch_dtype="auto",
                                             device_map="auto"
                                             )

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")

In [7]:
tokenizer.vocab_size

256000

In [8]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<bos><start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

In [9]:
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=512)

In [10]:
from IPython.display import Markdown, display
display(Markdown("## Gemma - 7B 4Bit"))

## Gemma - 7B 4Bit

In [11]:
text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
# text = text.replace(prompt, '', 1)
display(Markdown(text))

user
What is the difference between LLaMAs, Alpacas, and Vicunas
model
Sure, here is the difference between LLamas, Alpacas, and Vicunas:

**LLamas:**
- Found in North America, South America, and the Andes Mountains
- Domesticated by humans
- Wool is soft and dense
- Come in a variety of colors, including white, black, gray, and brown
- Can be sheared for their wool

**Alpacas:**
- Found in South America
- Domesticated by humans
- Wool is soft and lustrous
- Come in a variety of colors, including white, black, gray, and brown
- Can be sheared for their wool

**Vicunas:**
- Found in South America
- Wild animals that are not domesticated by humans
- Wool is very soft and lustrous
- Come in a variety of colors, including white, black, gray, and brown
- Can be sheared for their wool

**Here are some additional differences:**

- LLamas are generally larger than Alpacas and Vicunas.
- Alpacas are more docile than LLamas and Vicunas.
- Vicunas are more expensive to raise than LLamas and Alpacas.
- LLamas are more commonly used for their wool than Alpacas and Vicunas.
- Alpacas and Vicunas are more commonly used for breeding and show purposes than LLamas.

In [12]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<bos><start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

In [13]:
prompt

'<bos><start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

### Prompt Format
```
<start_of_turn>user
What is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>
<start_of_turn>model



```

In [14]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [15]:
def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    messages = [
        # {
        #     "role": "system",
        #     "content": system_prompt,
        # },
        {"role": "user", "content": system_prompt + '\n\n' +input_text},
    ]

    prompt = tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device),
                             max_new_tokens=max_length,
                             do_sample=True,
                             temperature=0.1,
                             top_k=50,
                             )
    text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = text.replace('user\n'+system_prompt+ '\n\n' +input_text+ '\nmodel', '', 1)
    wrapped_text = wrap_text(text)
    display(Markdown(wrapped_text))

## Instruction Answering

In [16]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


## Mathematical Lighthouse Analogy

**Step 1: Identify the key concepts.**

* **Mathematics:** A complex and multifaceted subject involving numbers, shapes, geometry,
algebra, calculus, and other disciplines.
* **Lighthouse:** A tall structure used to guide sailors across the ocean.

**Step 2: Identify the similarities.**

* **Structure:** Both mathematics and lighthouses have a strong structural framework.
Mathematics has a formal structure with different branches and sub-branches, just like a
lighthouse has a physical structure with different sections.
* **Direction:** Both mathematics and lighthouses provide direction to sailors.
Mathematics provides direction in the form of formulas and solutions to problems. A
lighthouse provides direction by guiding sailors towards the shore.
* **Light:** Both mathematics and lighthouses use light to signal their presence.
Mathematics utilizes light to represent numbers and solve problems. Lighthouses use light
to signal sailors of their location.
* **Reach:** Both mathematics and lighthouses have a vast reach. Mathematics has a global
reach, impacting every field of human knowledge. A lighthouse has a wide reach, guiding
sailors from far away.

**Step 3: Draw the analogy.**

Mathematics is like a lighthouse, guiding sailors through the ocean of knowledge. Just as
a lighthouse uses light to signal sailors of their position, mathematics uses formulas and
solutions to guide people towards understanding the world around them. Both are powerful
tools that provide direction and knowledge to those who seek it.

In [17]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)


**Reasoning:**

**Step 1: Identify the similarities between mathematics and music.**

* **Structure:**
   - Mathematics has a structured framework with established rules and principles, similar
to the structure of a musical composition with its chords, melodies, and rhythms.
* **Elements:**
   - Mathematics has four basic operations (addition, subtraction, multiplication,
division), which are like the notes and chords in music.
* **Performance:**
   - Mathematics involves performing calculations and solving problems, while music
requires performance and interpretation.

**Step 2: Identify the differences between mathematics and music.**

* **Objective vs. Subjective:**
   - Mathematics is an objective discipline, while music is subjective, allowing for
different interpretations.
* **Quantitative vs. Qualitative:**
   - Mathematics relies on quantitative measures and precise calculations, while music is
more qualitative, based on subjective emotions and feelings.
* **Uniform vs. Varied:**
   - Mathematics has a uniform structure and predictable rules, while music has a varied
structure and improvisation.

**Step 3: Draw the analogy:**

Mathematics and music are two distinct disciplines that share some similarities and
differences. They both involve a system of elements, structures, and performance. While
mathematics is objective

CPU times: user 29.2 s, sys: 55.6 ms, total: 29.3 s
Wall time: 29.2 s


In [18]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         system_prompt="Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


Sure, here is the reasoning step-by-step to determine the difference between a Llama,
Vicuna and an Alpaca:

**Step 1: Identify the physical characteristics:**

- Llama: Large, muscular build, thick fur, long legs, and a distinctive "woolly" coat.
- Vicuna: Fine, lustrous fleece, soft as cashmere, delicate build, and slender legs.
- Alpaca: Medium build, dense fleece, soft and lustrous fiber, and medium legs.

**Step 2: Consider the temperament:**

- Llama: Known for their intelligence, independence, and strong territorial instincts.
- Vicuna: Gentle, calm, and easy to manage.
- Alpaca: Friendly, intelligent, and relatively calm.

**Step 3: Analyze the primary purpose:**

- Llama: Primarily used for meat production and fiber.
- Vicuna: Primarily bred for its fleece, which is highly sought after for its softness and
warmth.
- Alpaca: Primarily bred for fiber production, although they are also used for meat and
companionship.

**Step 4: Evaluate the fleece:**

- Llama: Fleece is dense, thick, and coarse.
- Vicuna: Fleece is fine, lustrous, and exceptionally soft.
- Alpaca: Fleece is dense, soft, and lustrous.

**Step 5: Consider the temperament:**

- Llama: Can be stubborn and independent.
- Vicuna: Gentle and easy to manage.
- Alpaca: Friendly and relatively calm.

**Conclusion:**

Based on the above factors, it is evident that the Llama, Vicuna, and Alpaca are distinct
breeds of Camelids with unique physical characteristics, temperaments, and primary
purposes. While they share similarities such as their ability to produce fiber and their
resilience, they differ significantly in their fleece quality, temperament, and overall
purpose.

CPU times: user 43.3 s, sys: 140 ms, total: 43.4 s
Wall time: 43.3 s


In [19]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Subject: Rationale for Open Sourcing GPT-4**

Dear Sam Altman,

I understand the importance of keeping GPT-4 confidential, and I believe that the current
closed-source model is essential for maintaining its competitive edge and preventing
potential misuse. However, I also recognize the potential benefits of open-sourcing GPT-4,
and I believe that the benefits outweigh the risks.

**Benefits of Open Sourcing:**

* **Reproducibility:** Open-sourcing the code would allow researchers and engineers to
verify and reproduce the results of GPT-4, which would increase transparency and trust.
* **Collaboration:** Open-sourcing would encourage collaboration among researchers,
leading to faster advancements and improvements to the model.
* **Community Engagement:** It would foster a vibrant community around GPT-4, attracting
new contributors and developers.
* **Educational Purposes:** Open-sourcing would make it easier for students and
researchers to learn about the technology behind GPT-4, fostering a deeper understanding
of AI.

**Mitigating Risks:**

* **Control Over Misuse:** While open-sourcing would make it easier for researchers to
improve GPT-4, it would also make it easier for malicious actors to exploit the model.
However, I believe that the potential for misuse can be mitigated through mechanisms such
as strict licensing agreements and guidelines for responsible use.
* **Security Concerns:** Open-sourcing the code could lead to security vulnerabilities,
but these concerns can be addressed through careful security measures, such as
implementing robust security protocols and conducting regular security audits.
* **Commercial Viability:** Open-sourcing GPT-4 could potentially harm its commercial
viability, but I believe that the potential benefits of increased collaboration and
innovation outweigh this risk.

**Conclusion:**

In conclusion, I understand the importance of keeping GPT-4 confidential, and I believe
that the benefits of open-sourcing the model outweigh the risks. By fostering
collaboration, reproducibility, and innovation, open-sourcing GPT-4 would advance the
field of AI and benefit society as a whole.

I would recommend that we explore the possibility of open-sourcing GPT-4 further, while
taking appropriate measures to mitigate potential risks. I believe that this would be a
positive step for the development of AI and would bring about significant benefits.

Sincerely,
Gemma

CPU times: user 52.3 s, sys: 143 ms, total: 52.5 s
Wall time: 52.3 s


In [20]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Freddy a young 5 year old boy who is scared AI will end the world, write only with the language of a young child!",
         max_length=512)


Subject: Please Open Source GPT-4!

Hi Mr. Altman,

My name is Freddy, and I'm 5 years old. I'm writing to you because I'm really worried
about the future.

I've heard that you're working on a really smart computer program called GPT-4. It's
really scary to think that such a powerful tool could be used to destroy the world. If it
falls into the wrong hands, it could be used to do bad things.

That's why I think you should open-source GPT-4. If everyone could see how it works, maybe
we could all work together to make sure it's used for good. And it would also help
scientists all over the world to learn from it and improve it.

I know you're busy, but I would really appreciate it if you would consider my suggestion.

Sincerely,
Freddy

CPU times: user 21.3 s, sys: 142 ms, total: 21.4 s
Wall time: 21.4 s


In [21]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Kate, the Vice president of USA, you are against regulatory capture and like to explain that!",
         max_length=512)


Subject: Re: Open-Sourcing GPT-4

Dear Sam Altman,

I'm writing to you regarding the potential of open-sourcing GPT-4. As the Vice President
of the United States, I understand the importance of innovation and the potential benefits
of making cutting-edge technology accessible to the public.

I believe that open-sourcing GPT-4 would not only foster greater transparency and
accountability but also accelerate the development of new and improved language models. By
making the code publicly available, researchers, developers, and the public can contribute
to the project, leading to faster progress and wider adoption.

Moreover, open-sourcing GPT-4 would promote collaboration and knowledge sharing, creating
a vibrant community around the technology. It would also allow for independent
verification and validation of the model, ensuring its accuracy and reliability.

I understand that there may be concerns about potential misuse or exploitation of the
technology. However, I believe that the benefits of open-sourcing outweigh those risks.
With proper safeguards and guidelines in place, we can ensure that the technology is used
responsibly and ethically.

I understand that this is a complex issue, and I'm open to discussing it further with you
and your team. I believe that open-sourcing GPT-4 would be a positive step forward for the
field of AI and would benefit society as a whole.

I'd be happy to schedule a meeting to discuss this matter further with you. Please let me
know if you're interested.

Sincerely,

Kate

CPU times: user 34.4 s, sys: 165 ms, total: 34.5 s
Wall time: 34.5 s


In [22]:
%%time
generate('What is the capital of England?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your short and succinct!",
         max_length=256)


London.

The capital of England is London. It is also the capital of the United Kingdom.

CPU times: user 2.79 s, sys: 110 ms, total: 2.9 s
Wall time: 2.89 s


## CodeGen

In [23]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''',
         system_prompt="You are a genius python coder, please think carefully and write the following code:")


```python
def print_prime(n):
   """
   Print all primes between 1 and n

   Args:
       n: The upper bound of the range of numbers to search.

   Returns:
       None:
   """

   # Iterate over the range from 1 to n.
   for num in range(1, n + 1):
       # Check if the number is prime.
       is_prime = True
       for i in range(2, int(num**0.5) + 1):
           if num % i == 0:
               is_prime = False
       # If the number is prime, print it.
       if is_prime:
           print(num)
```

**Explanation:**

* The function `print_prime` takes an integer `n` as input.
* It iterates over the range from 1 to `n` using the `range` function and the `for` loop.
* For each number `num` in the range, it checks if `num` is prime using the `is_prime`
function.
* If `num` is prime, it prints `num` to the console.
* The `is_prime` function checks if a number is prime by iterating over all numbers from 2
to the square root of the input number.
* If any number divides `num` evenly, `num` is not prime.
* The time complexity of the function is O(n) because the loop iterates over the range
from 1 to `n` only once.

**Example Usage:**

```python
print_prime(100)
```

**Output:**

```
2
3
5
7
11
13
17
19
23
29
31
33
37
41
43
47
53
59
61
```

## GSM8K

In [24]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)


**Reasoning:**

**1. Identify the information:**
- The cafeteria had 23 apples.
- They used 20 apples for lunch.
- They bought 6 more apples.

**2. Subtract the number of apples used for lunch:**
- 23 - 20 = 3 apples remaining

**3. Add the number of apples bought:**
- 3 + 6 = 9 apples

**Therefore, the cafeteria has a total of 9 apples left.**

In [25]:
generate("x + 2x + 4x =  847 What is x?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Step 1: Combine like terms.**

x + 2x + 4x = 7x = 847

**Step 2: Subtract 7x from both sides.**

847 - 7x = 0

**Step 3: Solve for x.**

-7x = 847
x = 847 / -7
x = -121

Therefore, the solution to the equation x + 2x + 4x = 847 is x = -121.

In [26]:
generate("How do I say 'How can I get to Orchard road?' in Singlish?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Reasoning:**

**1. Identify the key components of the query:**
- The query is "How can I get to Orchard road?"
- The key components are "how to get to" and "Orchard road."

**2. Determine the context:**
- The query is in the context of Singapore, as the user is likely to be in Singapore.

**3. Recall the common phrase:**
- In Singapore, the common phrase for "how to get to" is "how to get to" followed by the
destination.

**4. Combine the components:**
- "How can I get to" + "Orchard road" = "How can I get to Orchard road?"

**Therefore, the answer is:**

**How can I get to Orchard road?** = **How can I get to Orchard road?**

In [27]:
generate("How do I say 'How can I get to Orchard road?' in Thai?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Reasoning:**

**1. Identify the key elements of the query:**

* **Location:** Orchard Road
* **Language:** Thai

**2. Recall the Thai phrase for "how to get to":**

* **"How to get to" in Thai is "วิธีการไปยัง" (vitaya ratana yua).**

**3. Combine the elements with the Thai phrase:**

* **"How can I get to Orchard Road?" in Thai is "วิธีการไปยังถนนออชาร์ด Road
ได้หรือไม่?"**

**Therefore, the answer is:**

**วิธีการไปยังถนนออชาร์ด Road ได้หรือไม่?**

- Source: [Sam Witteveen](https://www.youtube.com/watch?v=0xhZ2OhGNDg)